In [1]:
include("modle.jl")
using .Model
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
using LsqFit
println("KB=$kb")

KB=1.0


In [2]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 3.61

# 定义铜的FCC晶胞的基矢量
lattice_vectors = (Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))'

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,3,3,3)
fig=visualize_unitcell_atoms(cpcell)

In [3]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end

ct=5.0
interaction = Interaction(lj, Flj, ct, 0.1)

x=1:0.001:ct
y=interaction.cutenergy.(x)
lines(x,y)


In [4]:
println(cell_energy(cpcell,interaction))
println(cell_energy(cpcell,interaction))

-32.02658138108412
-32.02658138108412


In [5]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.02:2.5
en=0
fn=0
fl=Vector{Float64}([])
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,3,3,3)
    en=cell_energy0(cpcell,interaction,ifnormalize=true)
    push!(el,en)
    # fn=sum((cell_energy(cpcell,interaction)).^2)
    # push!(fl,fn)
    
end


In [6]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
vl=cl.^3*(cpcell.copy[1]*cpcell.copy[2]*cpcell.copy[3]);
# lines!(ax,cl,fl)
lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig

In [7]:
# 定义 Birch-Murnaghan 方程的函数
function birch_murnaghan(V, p)
    V0, B0, B0_prime, E0 = p
    eta = (V0 ./ V).^(2/3)  # 使用广播运算符 ./
    E = E0 .+ (9 * V0 * B0 / 16) .* ((eta .- 1).^3 .* B0_prime .+ (eta .- 1).^2 .* (6 .- 4 .* eta))
    return E
end

function BMfit(cl,el,p0::Vector{Float64}=[1200.0, -10.0, 4.0, -1200.0])
    fit = curve_fit(birch_murnaghan, vl, el, p0)

    # 拟合结果
    fitted_params = fit.param
    min_lattice_constant = (fitted_params[1]/cpcell.copy[1]/cpcell.copy[2]/cpcell.copy[3])^(1/3)
    println("Fitted parameters:")
    println("V0 = ", fitted_params[1])
    println("B0 = ", fitted_params[2])
    println("B0' = ", fitted_params[3])
    println("E0 = ", fitted_params[4])
    println("latticeconstant = ",min_lattice_constant )
    fig = Figure()
    fitfunc(x)=birch_murnaghan(x, fitted_params)
    # 创建一个 Axis，并设置坐标范围
    ax = Axis(fig[1, 1],xlabel="Volume", ylabel="Energy", title="Birch-Murnaghan EOS Fit"
              )
    x=LinRange(minimum(vl),maximum(vl),1000)
    scatter!(ax,vl,el,color=:red,label="Data")
    lines!(ax,x,fitfunc.(x),color=:blue,label="Fit")
    axislegend(ax, position=:rt)
    fig
    return min_lattice_constant,fig
end

BMfit (generic function with 2 methods)

In [8]:
_,fig=BMfit(vl,el)
fig

Fitted parameters:
V0 = 324.2275608215594
B0 = -10.578220561264184
B0' = 2.8089232566099525
E0 = -272.8134164568632
latticeconstant = 2.2899643516743198


In [7]:
function cell_forceij(cell::UnitCell, interaction::Interaction, i::Int, j::Int)
    cutoff = interaction.cutoff
    atomi = cell.atoms[i]
    atomj = cell.atoms[j]
    bd=atomj.bound
    cp = cell.copy
    cni = atomi.cn
    rij=atomj.position-atomi.position
    for k in 1:3
        rijk=rij[k]
        cpk=cp[k]
        if bd[k]==1
            continue
        end
        if rijk>cpk/2
                rij[k]=rijk-cpk
        elseif rijk<-cpk/2
                rij[k]=rijk+cpk
        end
    end
    rij=cell.lattice_vectors*rij
    nr=norm(rij)
    if nr>cutoff
        force=zeros(3)
    else
        force=interaction.cutforce(rij)
    end

    if any(isnan,force)
        println("Warning the same atoms of atom $i j=$j lead to the nan force,please check their cn")
    end
    return force
end

function cell_forcei(cell::UnitCell,interaction::Interaction,i::Int)
    forcei=zeros(3)
        for j in 1:length(cell.atoms)
            if i!=j
                forcei+=cell_forceij(cell,interaction,i,j)
            end
        end
    return forcei
end



function force_tensor(cell::UnitCell,interaction::Interaction)
    tensor=zeros(3,3)
    v=cell.Volume
    for a in 1:3
        for b in 1:3
            for i in 1:length(cell.atoms)
                forcei=cell_forcei(cell,interaction,i)
                atom=cell.atoms[i]
                tensor[a,b]+=forcei[a]*atom.position[b]/2+atom.momentum[a]*atom.momentum[b]/atom.mass
            end
        end
    end
    return tensor./v
end


force_tensor (generic function with 1 method)

In [72]:

###MD
function dUdV_default(r::Vector{Float64},v::Float64)
    return 0.0
end

function pressure_int(cell::UnitCell,interaction::Interaction,dUdV::T=dUdV_default) where T
    v=cell.Volume
    Pint=0.0
    for i in 1:length(cell.atoms)
        atom=cell.atoms[i]
        pm=atom.momentum
        ri=atom.position
        fi=cell_forcei(cell,interaction,i)
        Pint+=dot(pm,pm)/atom.mass+dot(ri,fi)-3*v*dUdV(atom.position,v)   
    end
    return Pint/v/3
    
end


struct Thermostat
    T::Float64  # 目标温度
    Q::Float64  # 热浴质量
    Rt::Float64  # 热浴变量 (friction coefficient)
    Pt::Float64  # 热浴动量
end
struct Barostat
    Pe::Float64  # 目标压力
    V::Float64  # 系统体积
    W::Float64  # 压力浴质量
    Pv::Float64  # 压力浴动量
end


function Hz(z::Vector{Float64},cell::UnitCell,interaction::Interaction,thermostat::Thermostat,barostat::Barostat,dUdV::T=dUdV_default) where T
    global kb
    dim=3*length(cell.atoms)+2
    Hz=zeros(dim*2)
    natom=length(cell.atoms)
    v=cell.Volume
    W=barostat.W
    Q=thermostat.Q
    temp=thermostat.T
    Pe=barostat.Pe
    Pint=pressure_int(cell,interaction,dUdV)
    addp=sum([dot(atom.momentum,atom.momentum)/atom.mass for atom in cell.atoms])
    for i in 1:natom
        atom=cell.atoms[i]
        mi=atom.mass
        Hz[3*i-2]=1/mi+z[2*dim]*z[3*i-2]/W
        Hz[3*i-1]=1/mi+z[2*dim]*z[3*i-1]/W
        Hz[3*i]=1/mi+z[2*dim]*z[3*i]/W
        fi=-cell_forcei(cell,interaction,i)
        Hz[dim+3*i-2]=fi[1]-(1+1/natom)*z[2*dim]/W*z[dim+3*i-2]-z[2*dim-1]*z[2*dim]/Q
        Hz[dim+3*i-1]=fi[2]-(1+1/natom)*z[2*dim]/W*z[dim+3*i-1]-z[2*dim-1]*z[2*dim]/Q
        Hz[dim+3*i]=fi[3]-(1+1/natom)*z[2*dim]/W*z[dim+3*i]-z[2*dim-1]*z[2*dim]/Q
        Hz[dim]=3*z[dim]*z[2*dim]/W
        Hz[2*dim]=3*z[dim]*(Pint-Pe)+1/natom*addp-z[2*dim-1]*z[2*dim]/Q
        z[dim-1]=z[2*dim-1]/Q
        z[2*dim-1]=addp+z[2*dim]^2/W-(3*natom+1)*kb*temp
    end

    return Hz
end

function symplectic_matrix(n::Int)
    I = Matrix{Float64}(I, n, n)  # 创建 n x n 的单位矩阵
    O = zeros(Float64, n, n)      # 创建 n x n 的零矩阵
    J = [O I; -I O]               # 使用块矩阵构建辛矩阵
    return J
end

symplectic_matrix (generic function with 1 method)

In [73]:
thermostat = Thermostat(300.0, 1000.0, 0.1, 0.0)
barostat=Barostat(0.0,cell.Volume,1000.0,0.0)

Barostat(0.0, 15.625, 1000.0, 0.0)

In [74]:
rl=[atom.position for atom in cell.atoms];
pl=[atom.momentum for atom in cell.atoms];

In [75]:
rl=[atom.position for atom in cpcell.atoms];
pl=[atom.momentum for atom in cpcell.atoms];
z=vcat(vcat(rl...),thermostat.Rt,barostat.V,vcat(pl...),thermostat.Pt,barostat.Pv);

In [77]:

# 定义 symplectic_matrix 函数
function symplectic_matrix(n::Int)
    I = Matrix{Float64}(I, n, n)  # 创建 n x n 的单位矩阵
    O = zeros(Float64, n, n)      # 创建 n x n 的零矩阵
    J = [O I; -I O]               # 使用块矩阵构建辛矩阵
    return J
end

# 定义 RK3 step
function RK3_step(z::Vector{Float64},dt::Float64,cell::UnitCell, interaction::Interaction, thermostat::Thermostat, barostat::Barostat, dUdV::T = dUdV_default) where T
        k1=Hz(z,cell,interaction,thermostat,barostat,dUdV)
        k2=Hz(z+dt/2*k1,cell,interaction,thermostat,barostat,dUdV)
        k3=Hz(z-dt*k1+2*dt*k2,cell,interaction,thermostat,barostat,dUdV)
        newz=z+dt/6*(k1+4*k2+k3)
    return newz
end


RK3_step (generic function with 4 methods)

In [78]:
for i in 1:100
    z=RK3_step(z,Hz,0.01,cpcell,interaction,thermostat,barostat)
end

In [80]:
Hz(z,cpcell,interaction,thermostat,barostat)

1036-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   0.0
 NaN

In [82]:
cell_forcei(cpcell,interaction,1)

3-element Vector{Float64}:
 -0.009180448197762977
 -0.009180448197763032
 -0.009180448197763081

In [83]:
visualize_unitcell_atoms(cpcell)